# Neural Turing Machine learning parenthesis matching problem

#### Imports

In [1]:
import numpy as np

import tensorflow as tf

from random import randint

from pythonds.basic.stack import Stack

from numpy.random import choice

C:\Users\sankho.roy\Application\Conda\envs\Updated_pip_10\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Helper Functions

This function checks if parenthesis are balanced or not by Naive Old School Algorithm

In [2]:
def parChecker(symbolString):
    s = Stack()
    balanced = True
    index = 0
    while index < len(symbolString) and balanced:
        symbol = symbolString[index]
        if symbol == "(":
            s.push(symbol)
        else:
            if s.isEmpty():
                balanced = False
            else:
                s.pop()

        index = index + 1

    if balanced and s.isEmpty():
        return True
    else:
        return False


### Dataset Generator Functions

- dataGenerator() generate 50% balanced and 50% unbalanced parenthesis. *We are going to talk about **delta** very soon.*

In [3]:
def dataGenerator(length = 10,delta = 2.5):
    res = ''
    left = 0
    
    for i in range(length):
        prob = []
        if left > 0:
            p = np.abs(left)/(np.abs(left)+delta)
            prob.append(p)
            prob.append(1-p)
        else :
            p = np.abs(left)/(np.abs(left)+delta)
            prob.append(1-p)
            prob.append(p)
            
        draw = choice([0,1], 1, p=prob)
        if draw == 0 :
            res += '('
            left += 1
        else :
            res += ')'
            left -= 1
                 
    return res,parChecker(res)
    

- dataGeneratorNotBiased() generates  parenthesis generated from a field where probability of selecting ( or ) is 50%
 so , balanced and unbalanced parenthesis sequence is not 50% and 50%. It is usually skewed.
 For n parenthesis  it is <br><br> **Prob(parenthesis Sequence is Matched) ~ CatalanNumber(n)/(2^n)**
 <br><br> and  **Prob(parenthesis Sequence is Not Matched) ~  1 - CatalanNumber(n)/(2^n)**

In [4]:
def dataGeneratorNotBiased(length = 10,delta = 2.5):
    res = ''
    left = 0
    
    for i in range(length):            
        draw = choice([0,1], 1, p=[0.5,0.5])
        if draw == 0 :
            res += '('
            left += 1
        else :
            res += ')'
            left -= 1
                 
    return res,parChecker(res)

### Tokenizers for brakets and labels
( = [0,1] <br>
) = [1,0] <br>

True or Matched parenthesis Sequence = [1,0] <br>
False or Unmatched parenthesis Sequence = [1,0]

In [5]:
def tokenizer(braket_string):
    inp_list = []
    dic = {
      '(':[0,1],
      ')':[1,0]}
    
    brakets = list(braket_string)
    
    for braket in brakets:
        inp_list.append(dic[braket])
    inp = np.array(inp_list)
    return inp

def tokenizer_label(output):
    inp_list = []
    dic = {
        True:[1,0],
      False:[0,1]
    }
        
    inp_list.append(dic[output])
    inp = np.array(inp_list)
    return inp

### InputMaker functions for dataGenerator()  and dataGeneratorNotBiased() function

In [6]:
def inputMakerNotBiased():
    x,y = dataGeneratorNotBiased(length=seq_length,delta=2.5)
    print(x,y)
    generated_inp =  x 
    X = tokenizer(generated_inp)
    X = X.T
    Y = tokenizer_label(y)
    Y = Y.T
    X = np.array(X,dtype='float32')
    Y = np.array(Y,dtype='float32')
    return X,Y

In [7]:
def inputMaker():
    x,y = dataGenerator(length=seq_length,delta=2.5)
    print(x,y)
    generated_inp =  x 
    X = tokenizer(generated_inp)
    X = X.T
    Y = tokenizer_label(y)
    Y = Y.T
    X = np.array(X,dtype='float32')
    Y = np.array(Y,dtype='float32')
    return X,Y


### What is *Delta* ?

- If you have probability of choosing either ( or ) is 50% then for n parenthesis  it is <br><br> *Prob(parenthesis Sequence is Matched) ~ CatalanNumber(n)/(2^n)*<br> and  *Prob(parenthesis Sequence is Not Matched) ~  1 - CatalanNumber(n)/(2^n)*.<br><br>

- So, we defined new function **dataGenerator()** which forcefully creates *Prob(parenthesis Sequence is Matched) ~ 0.5*
  and *Prob(parenthesis Sequence is Not Matched) ~ 0.5* by choosing suitable **delta**. <br>
  e.g for  sequence length  = 100 - delta = 4.5 <br>
    for sequence length = 16 - delta = 3.1  <br>
    for sequence length = 400 - delta=6 <br>
    and for  sequence length = 1000 - delta=6.4 <br><br>
    
- ***This cell below to experiment and check for which delta gives you 50%, 50% probability.***

In [8]:
true_count = 0
false_count = 0
for i in range(100):
    x,y = dataGenerator(length=10,delta=2.5) 
    
    if y :
        true_count += 1
    else:
        false_count += 1
        
print((true_count/(true_count+false_count)))
print((false_count/(true_count+false_count)))

0.56
0.44


## The Neural  Network 

<img src="ntm.png">

### Hyperparameters

In [9]:
input_size = 2
output_size = 2

In [10]:
memory_bits = 2
memory_length = 10
shift_length = 4
seq_length = 10
controller_output_size = 2

In [11]:
controller_input_len = input_size + memory_bits

In [12]:
controller_output_len = memory_bits + memory_bits + 1 + shift_length + 1 + memory_bits + memory_bits + controller_output_size

weight_matrix is our **RAM** in NTM cell and controller neural net is processor of NTM cell.

In [13]:
weight_matrix = tf.fill([memory_bits,memory_length],1e-6)

### One step() function operates on a single NTM cell 

In [14]:
def step(i,x,read_vector):
    global weight_matrix
    x = tf.concat([x,read_vector],0) 
    controller_input = tf.reshape(x,[controller_input_len,1])
    controller_output = tf.matmul(controller_weight,controller_input,transpose_a=True) + controller_bias
    controller_output = tf.nn.tanh(controller_output)
    
    controller1_output = tf.matmul(controller1_weight,controller_output,transpose_a=True) + controller1_bias
    controller1_output = tf.nn.tanh(controller1_output)
    
    partition = tf.constant([0]*(memory_bits) + [1]*(memory_bits) + [2]*(1) + [3]*(shift_length) + [4]*(1) + [5]*(memory_bits) + [6]*(memory_bits) + [7]*(controller_output_size)) 
    
    (beta, k , g , s , gamma , erase_vector , add_vector, output) = tf.dynamic_partition(controller1_output , partition,8)
    
    k_norm = tf.nn.l2_normalize(k,axis=0)
    mat_norm = tf.nn.l2_normalize(k,axis=0)
    
    w_c = tf.nn.softmax(tf.matmul(mat_norm,k_norm,transpose_a=True)*beta)
    w_g = tf.add((w_c*g) , (g*w_vector[i])) #(1-g)
    s = tf.reshape(s,[shift_length,1])

    
#     s = tf.concat([s[:, shift_length + 1],
#                        tf.zeros([s.get_shape()[0], memory_bits - (shift_length * 2 + 1)]),
#                        s[:, -shift_length:]], axis=1)
#     t = tf.concat([tf.reverse(s, axis=[1]), tf.reverse(s, axis=[1])], axis=1)
#     s_matrix = tf.stack(
#             [t[:, memory_bits - i - 1:memory_bits * 2 - i - 1] for i in range(memory_bits)],
#             axis=1
#         )
#     w_tilda = tf.reduce_sum(w_g * s_matrix, axis=2) 
        
        
        
    w_tilda = w_g   #to be done
    w = tf.norm(tf.pow(w_tilda,gamma),keepdims=True,axis=1)
#     print(w)
    tf.assign(w_vector[i+1],w)
    
    erase_vector = tf.reshape(erase_vector,[memory_bits,1])
    erase_vector = tf.nn.softmax(erase_vector,axis=1)
    erase_matrix = tf.matmul(erase_vector,w,transpose_b=True)
    ones_matrix = tf.ones_like(erase_matrix)
    
#     print(erase_matrix)
    weight_matrix = weight_matrix*(ones_matrix - erase_matrix)
#     print(weight_matrix)
    
    add_vector = tf.reshape(add_vector,[memory_bits,1])
    add_vector = tf.nn.softmax(add_vector,axis=1)
    add_matrix = tf.matmul(add_vector,w,transpose_b=True)
    weight_matrix = weight_matrix + add_matrix
#     print(add_matrix)
    
    read_head = tf.matmul(weight_matrix,w_vector[i+1])
    

    output = tf.nn.softmax(output,axis=0) #if added no training is done ...
    return read_head,output

### Tensorflow Placeholder and Variables declared here  

In [15]:
x = tf.placeholder(tf.float32,[seq_length,input_size,1])
y = tf.placeholder(tf.float32,[output_size,1])

read_vector = tf.Variable(tf.truncated_normal([seq_length+1,memory_bits,1]))
w_vector = tf.Variable(tf.zeros([seq_length+1,memory_length,1]))

controller_weight = tf.Variable(tf.truncated_normal([controller_input_len,128]))
controller_bias = tf.Variable(tf.truncated_normal([128,1]))

controller1_weight = tf.Variable(tf.truncated_normal([128,controller_output_len]))
controller1_bias = tf.Variable(tf.truncated_normal([controller_output_len,1]))

output_weight = tf.Variable(tf.truncated_normal([controller_output_size,output_size]))
output_bias = tf.Variable(tf.truncated_normal([output_size,1]))

### do_it() function runs whole NTM at once and shows results

In [16]:
number_of_train_samples = 400
number_of_test_samples = 500

epochs_in_training = 1000

In [17]:
def do_it(x):
    
    for i,val in enumerate(tf.unstack(x,axis=0)):
        c_output, output = step(i,val,read_vector[i])
        tf.assign(read_vector[i+1], c_output)
        if i == seq_length-1:
            out = output
            out = tf.matmul(output_weight,out,transpose_a=True) + output_bias
            out = tf.nn.sigmoid(out)
    
    return out
 

with tf.Session() as sess:
    exp = do_it(x)
    loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=exp, labels=y))
    optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss)
    
    
    init = tf.initialize_all_variables()
    sess.run(init)
    
    
    
    for num_sample in range(number_of_train_samples):
        print(num_sample)
        inp,target = inputMaker()
#         exp = do_it(x)
#         loss = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=exp, labels=y))
#         optimizer = tf.train.AdamOptimizer(learning_rate=0.00001).minimize(loss)
        
        
        inp = np.reshape(inp,(seq_length,2,1))
        weight_matrix = tf.fill([memory_bits,memory_length],0.00)
        for i in range(epochs_in_training):
            
            p = sess.run([exp],feed_dict={x:inp,y:target})
            a,_,c = sess.run([exp,optimizer,loss],feed_dict={x:inp,y:target})
#             print('loss in iteration ',i,c)
        print('____________________________________________________________________________________________________')
        print('sample no.',num_sample)
        print('....................................................................................................')
        print('trained true : ',target)
        print('trained predicted',a)

        print('___________________________________________________________________________________________________')
        print('\n')
    
    
                # Testing Starts here ...      
    
    
    print('\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n Testing ...\n')
    
    correct_prediction = 0        
    for num_sample in range(number_of_test_samples):
        inp,target = inputMaker()
        
        
        weight_matrix = tf.fill([memory_bits,memory_length],0.00)
        inp = np.reshape(inp,(seq_length,2,1))
        

        a = sess.run([exp],feed_dict={x:inp,y:target})
#             print('loss in iteration ',i,c)
        print('____________________________________________________________________________________________________')
        print('sample no.',num_sample)
        print('....................................................................................................')
        print('trained true : ',target)
        print('trained predicted',a)
        if np.argmax(a[0],axis=0) == np.argmax(target,axis=0):
            print('Correct predicted '+u'\u2713')
            correct_prediction += 1
        print('___________________________________________________________________________________________________')
        print('\n')

Instructions for updating:
Use `tf.global_variables_initializer` instead.
0
()()()(((( False
____________________________________________________________________________________________________
sample no. 0
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [[0.72572696]
 [0.6605014 ]]
___________________________________________________________________________________________________


1
(())()()() True
____________________________________________________________________________________________________
sample no. 1
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [[0.7658384 ]
 [0.59079224]]
___________________________________________________________________________________________________


2
()()()()() True
_____________________________________________________________________________________

____________________________________________________________________________________________________
sample no. 20
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [[0.71856236]
 [0.627895  ]]
___________________________________________________________________________________________________


21
()()()()() True
____________________________________________________________________________________________________
sample no. 21
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [[0.771071  ]
 [0.53929573]]
___________________________________________________________________________________________________


22
(())()(()) True
____________________________________________________________________________________________________
sample no. 22
...........................................................

____________________________________________________________________________________________________
sample no. 40
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [[0.71779364]
 [0.58610123]]
___________________________________________________________________________________________________


41
()()()()(( False
____________________________________________________________________________________________________
sample no. 41
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [[0.71365327]
 [0.5897107 ]]
___________________________________________________________________________________________________


42
()()((()() False
____________________________________________________________________________________________________
sample no. 42
.........................................................

____________________________________________________________________________________________________
sample no. 60
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [[0.7378513 ]
 [0.50137967]]
___________________________________________________________________________________________________


61
()()()()() True
____________________________________________________________________________________________________
sample no. 61
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [[0.74076074]
 [0.49601936]]
___________________________________________________________________________________________________


62
()()()(((( False
____________________________________________________________________________________________________
sample no. 62
..........................................................

____________________________________________________________________________________________________
sample no. 80
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [[0.7136772 ]
 [0.49157825]]
___________________________________________________________________________________________________


81
(()()()((( False
____________________________________________________________________________________________________
sample no. 81
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [[0.6321188]
 [0.5988502]]
___________________________________________________________________________________________________


82
(())()()(( False
____________________________________________________________________________________________________
sample no. 82
...........................................................

...

____________________________________________________________________________________________________
sample no. 412
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [array([[0.45747128],
       [0.3418797 ]], dtype=float32)]
___________________________________________________________________________________________________


()()()(()) True
____________________________________________________________________________________________________
sample no. 413
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [array([[0.45747128],
       [0.3418797 ]], dtype=float32)]
Correct predicted ✓
___________________________________________________________________________________________________


()()()()() True
________________________________________________________________________________________________

____________________________________________________________________________________________________
sample no. 433
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [array([[0.45747128],
       [0.3418797 ]], dtype=float32)]
Correct predicted ✓
___________________________________________________________________________________________________


()()()()() True
____________________________________________________________________________________________________
sample no. 434
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [array([[0.45747128],
       [0.3418797 ]], dtype=float32)]
Correct predicted ✓
___________________________________________________________________________________________________


()()()(()) True
____________________________________________________________________________

____________________________________________________________________________________________________
sample no. 451
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [array([[0.25897634],
       [0.5617023 ]], dtype=float32)]
Correct predicted ✓
___________________________________________________________________________________________________


(((()(((() False
____________________________________________________________________________________________________
sample no. 452
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [array([[0.45747128],
       [0.3418797 ]], dtype=float32)]
___________________________________________________________________________________________________


()(())()() True
_______________________________________________________________________________________________

____________________________________________________________________________________________________
sample no. 469
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [array([[0.25897634],
       [0.5617023 ]], dtype=float32)]
Correct predicted ✓
___________________________________________________________________________________________________


()()(()((( False
____________________________________________________________________________________________________
sample no. 470
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [array([[0.25897634],
       [0.5617023 ]], dtype=float32)]
Correct predicted ✓
___________________________________________________________________________________________________


()()()(()) True
___________________________________________________________________________

____________________________________________________________________________________________________
sample no. 487
....................................................................................................
trained true :  [[1.]
 [0.]]
trained predicted [array([[0.45747128],
       [0.3418797 ]], dtype=float32)]
Correct predicted ✓
___________________________________________________________________________________________________


()()()((() False
____________________________________________________________________________________________________
sample no. 488
....................................................................................................
trained true :  [[0.]
 [1.]]
trained predicted [array([[0.45747128],
       [0.3418797 ]], dtype=float32)]
___________________________________________________________________________________________________


((()(((()) False
______________________________________________________________________________________________

### Check the accuracy here :

In [18]:
print(correct_prediction/number_of_test_samples)

0.826


In [19]:
print('for {} brakets accuracy is {}'.format(seq_length,(correct_prediction/number_of_test_samples)))

for 10 brakets accuracy is 0.826


for 20 brakets accuracy is 0.87

for 100 accuracy is 0.886

for 400 accuracy is 0.936

for 1000 accuracy is 0.948